In [42]:
import pandas as pd
import numpy as np
import preprocess
from sklearn.linear_model import LogisticRegression

In [39]:
url = 'https://raw.githubusercontent.com/probml/pmtkdata/master/spamData/spam.data'
data = pd.read_table(url, header = None, sep = ' ')

In [40]:
y = data[57].values
X = data[range(57)]
del data

In [38]:
# preprocess.scale(X)
# np.log(X + 0.1)
# 1 * (X > 0)